### 0. Imprts 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import matplotlib.dates as mdates

import folium 
import missingno as msno
import geopy.distance
from sklearn.preprocessing import Imputer
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from nltk.corpus import stopwords 

%matplotlib inline

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import r2_score

# extra setting to show all columns in descriptive statistics
pd.set_option('display.max_columns', 500)

### 1. Data load

In [ ]:
calendar_all = pd.read_csv("calendar_all.csv", parse_dates=['date'])

listings = pd.read_csv("listings.csv")

detailed_listings = pd.read_csv("detailed_listings.csv", low_memory = False)

neighbourhoods = pd.read_csv("neighbourhoods.csv")

reviews = pd.read_csv("reviews.csv")

detailed_reviews = pd.read_csv("detailed_reviews.csv")

### 1. Get to know data

#### 1.1.Are where any duplicates? 

In [ ]:
def duplicates_counts(df, name):
    print("{} dataset has {} rows and {} columns and it contains {} duplicates".format(name, df.shape[0], df.shape[1], df.duplicated().sum()))

In [ ]:
duplicates_counts(listings, "listings")
duplicates_counts(detailed_listings, "detailed_listings")
duplicates_counts(neighbourhoods, "neighbourhoods")
duplicates_counts(reviews, "reviews")
duplicates_counts(detailed_reviews, "detailed_reviews")

Calculations show that there are duplicates in reviews dataset. But taking into account business logic, that the same day multiple users can evaluate their stay, it might be not true. I randomly selected one of duplicated values from review dataset and checked in dateiled_reviews dataframe

In [ ]:
reviews[reviews.duplicated()].head(5)

In [ ]:
detailed_reviews[(detailed_reviews['listing_id'] == 61714) & (detailed_reviews['date'] == "2017-05-29")]

Different users evaluate their stay. Unique of these values describes reviewer_id, reviewer_name and comment. 

#### 1.2. Is there many missing data?

In [ ]:
def identify_missing_values(df):
    msno.matrix(df)
    print(df.columns[df.isnull().any()])
    print(df.isnull().mean().sort_values(ascending = False).head(5))
    

In [ ]:
identify_missing_values(listings)

There are some missing values in last revew and reviews per month columns. This might be a result of new listings in Airbnb platform. Maybe listings are too new to get any feedback. Legend in the right indicate rows with the lowest and highest number of missing values.

In [ ]:
identify_missing_values(detailed_listings)

Detailed_listings dataset had multiple columns with missing values. Columns with all null values: thumbnail_url, host_acceptance_rate, xl_picture_url          

In [ ]:
msno.heatmap(detailed_listings)

The missingno correlation heatmap measures nullity correlation. Where correlation = 1 means, that if one variable appears, the other one definetely appears. That might show us the structure of the data. Values marked as < 1, shows that there are some rows where not both records are presented

In [ ]:
msno.dendrogram(detailed_listings)

Dendogram shows exact "causality" and relationship between columns. How each column is realted with other ones. 

In [ ]:
identify_missing_values(reviews)

As could be expected, reviews dataset has all values. As row is inserted after user submit a review. 

In [ ]:
identify_missing_values(detailed_reviews)

Only few rows in detailed_reviews dataset have missing values

In [ ]:
identify_missing_values(neighbourhoods)

No issues in neighbourhoods dataset. 

In [ ]:
identify_missing_values(calendar_all)

#### 1.3. Dataset column types

In [ ]:
calendar_all.dtypes

Calender date value type changed to datetime, price and adjusted price should be changed to int value in order to user this value for calculations.

In [ ]:
calendar_all['price'] = calendar_all.price.str.replace("[$, ]", "").astype("float")
calendar_all['adjusted_price'] = calendar_all.adjusted_price.str.replace("[$, ]", "").astype("float")


In [ ]:
listings.dtypes

Detailed_listings dataset has multiple columns with icorrect type. Some date values, prices stored as text. 

In [ ]:
detailed_listings.head(1)

In [ ]:
listingsTypeDict = dict(detailed_listings.dtypes)
listingsTypeDict

In [ ]:
# price values convertion
detailed_listings['price'] = detailed_listings.price.str.replace("[$, ]", "").astype("float")
detailed_listings['cleaning_fee'] = detailed_listings.cleaning_fee.str.replace("[$, ]", "").astype("float")
detailed_listings['weekly_price'] = detailed_listings.weekly_price.str.replace("[$, ]", "").astype("float")
detailed_listings['monthly_price'] = detailed_listings.monthly_price.str.replace("[$, ]", "").astype("float")

# response rate to value
detailed_listings['host_response_rate'] = detailed_listings['host_response_rate'].str.replace(r'%', r'.0').astype('float') / 100.0


# date values converted to date
detailed_listings['host_since']= pd.to_datetime(detailed_listings['host_since']) 
detailed_listings['calendar_last_scraped']= pd.to_datetime(detailed_listings['calendar_last_scraped']) 
detailed_listings['last_review']= pd.to_datetime(detailed_listings['last_review']) 
detailed_listings['last_scraped']= pd.to_datetime(detailed_listings['last_scraped']) 


In [ ]:
reviews.dtypes

In [ ]:
detailed_reviews.dtypes

In [ ]:
detailed_reviews['date']= pd.to_datetime(detailed_reviews['date']) 

#### 1.4 Fill missing data

I am going to fill missing values of numeric columns, especially ones, which I am going to use in the analysis

In [ ]:
calendar_all.dtypes

Columns with more than 30% of missing values I am going to drop. As this would be too much biased to fill values which are not known

In [ ]:
missing_values = detailed_listings.isnull().mean().sort_values(ascending = False).where(lambda x : x> 0.3).dropna().to_frame().reset_index().rename(columns={"index": "name", 0: "missing"})

In [ ]:
missing_value_cols = missing_values.name.values

In [ ]:
detailed_listings = detailed_listings.drop(missing_value_cols, axis = 1)

In [ ]:
# If superhost value is nan, then mark as not superhost
# detailed_listings.at[detailed_listings['host_is_superhost'].isnull(), 'review_scores_rating'] = 'f'
detailed_listings['host_is_superhost'].fillna("f", inplace=True)

# If bathrooms value is nan, then replace with 0
detailed_listings['bathrooms'].fillna(0, inplace=True)

# If bedrooms value is nan, then median
detailed_listings['bedrooms'].fillna((detailed_listings['bedrooms'].median()), inplace=True)

# If beds value is nan, then mean, because it should be at least 1
detailed_listings['beds'].fillna((detailed_listings['beds'].mean()), inplace=True)

# If no cleaning fee provided, then assume to be 0
detailed_listings['cleaning_fee'].fillna(0, inplace=True)

# If no value for deposit, then assume to be 0
detailed_listings['security_deposit'].fillna(0, inplace=True)

# If no identification provided, then assume to be false - f
detailed_listings['host_identity_verified'].fillna("f", inplace=True)

# If no photo provided, then assume to be false - f
detailed_listings['host_has_profile_pic'].fillna("f", inplace=True)

# If the property was not booked or there were no evaluation, I am filling values with 0
detailed_listings['review_scores_rating'].fillna(0, inplace=True)
detailed_listings['review_scores_accuracy'].fillna(0, inplace=True)
detailed_listings['review_scores_cleanliness'].fillna(0, inplace=True)
detailed_listings['review_scores_checkin'].fillna(0, inplace=True)
detailed_listings['review_scores_communication'].fillna(0, inplace=True)
detailed_listings['review_scores_location'].fillna(0, inplace=True)
detailed_listings['review_scores_value'].fillna(0, inplace=True)
detailed_listings['reviews_per_month'].fillna(0, inplace=True)

detailed_listings['host_response_rate'].fillna(0, inplace=True)
detailed_listings['host_listings_count'].fillna(0, inplace=True)
detailed_listings['host_total_listings_count'].fillna(0, inplace=True)


# text columns, if not provided, then ""
detailed_listings['neighborhood_overview'].fillna("", inplace=True)
detailed_listings['house_rules'].fillna("", inplace=True)
detailed_listings['space'].fillna("", inplace=True)
detailed_listings['host_response_time'].fillna("", inplace=True)
detailed_listings['neighbourhood'].fillna("", inplace=True)
detailed_listings['summary'].fillna("", inplace=True)
detailed_listings['state'].fillna("Veneto", inplace=True)
detailed_listings['zipcode'].fillna("", inplace=True)
detailed_listings['description'].fillna("", inplace=True)
detailed_listings['host_location'].fillna("Venice, Veneto, Italy", inplace=True)
detailed_listings['city'].fillna("Venezia", inplace=True)
detailed_listings['host_thumbnail_url'].fillna("", inplace=True)
detailed_listings['host_name'].fillna("", inplace=True)
detailed_listings['smart_location'].fillna("Venice, Italy", inplace=True)
detailed_listings['host_picture_url'].fillna("", inplace=True)


#### 1.5. Outliers detection

After some string type columns were converted to integers, look at decriptive statistics, to detect exceptional values 

In [ ]:
calendar_all.describe()

Seems unreasonale maximum price 1430 for 1 night

In [ ]:
listings.describe()

Listings maximum price = 8459, while 75% is 150. That means there are outlers. Number of counts is quite difficult to test if 126 is the right maximum number. 

In [ ]:
detailed_listings.describe()

maximum number of bathrooms value = 105, 75 percentile = 1.5, need outlier detection;

price maximum value = 8459, 75 percentile = 150, need outlier detection;

cleaning fee maximum value = 2222, 75 percentile = 60, need outlier detection;

minimum nights maximum value = 107, while 75 percentile = 3

In [ ]:
# def remove_outliers(df):
#     num_train = df.select_dtypes(include=['number'])
#     cat_train = df.select_dtypes(exclude=['number'])
#     Q1 = num_train.quantile(0.25)
#     Q3 = num_train.quantile(0.75)
#     IQR = Q3 - Q1
#     idx = ~((num_train < (Q1 - 1.5 * IQR)) | (num_train > (Q3 + 1.5 * 
#     IQR))).any(axis=1)
#     train_cleaned = pd.concat([num_train.loc[idx], cat_train.loc[idx]], axis=1)
#     return train_cleaned

In [ ]:
col_list = ['price', 'bathrooms', 'cleaning_fee']

def remove_outliers(df, col_list):
    num_train = df[col_list]
    cat_cols = [col for col in df.columns if col not in col_list]    
    cat_train = df[cat_cols]
    Q1 = num_train.quantile(0.25)
    Q3 = num_train.quantile(0.75)
    IQR = Q3 - Q1
    idx = ~((num_train < (Q1 - 1.5 * IQR)) | (num_train > (Q3 + 1.5 * IQR))).any(axis=1)
    train_cleaned = pd.concat([num_train.loc[idx], cat_train.loc[idx]], axis = 1)
    return train_cleaned

In [ ]:
detailed_listings = remove_outliers(detailed_listings, col_list)

#### 1.6. Correlations

Because it is hardly to believe that all variables follow Normal distribution, I calculate Spearman correlation coefficient

In [ ]:
def calculate_correlation(df):
    corr = df.corr(method="spearman")
    plt.figure(figsize=(16, 12))
    sns.heatmap(corr[(corr >= 0.8) | (corr <= -0.8)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

In [ ]:
calculate_correlation(detailed_listings)

In [ ]:
calculate_correlation(listings)

### 2. Descriptive analysis/ add new columns

### 2.1 Create new features

#### 2.1.1 How far listing is from city center? Venice coordinates taken from: https://www.planetware.com/tourist-attractions-/venice-i-vn-v.htm 

In [ ]:
lat, long = 45.434307, 12.339159

In [ ]:
listings['central_lat'] = lat
listings['central_long'] = long


In [ ]:
listings['distance_km'] = listings.apply(
    (lambda row: geopy.distance.geodesic(
        (row['latitude'], row['longitude']),
        (row['central_lat'], row['central_long'])
    ).miles),
    axis=1
).round(2)

In [ ]:
listings.hist(column='distance_km')

In [ ]:
cut_bins = [0, 0.5, 1, 1.5, 2, 3, 5, 10]
listings['listings_km_bins'] = pd.cut(listings['distance_km'], bins = cut_bins)

In [ ]:
listings['listings_km_bins'].value_counts()

#### 2.1.2. How many days listing in platform?

Calculated the maximum and minimum day of last scraped. The first date is 2019-11-09, the last one 2019-12-06. This let me understand dates variability. 

In [ ]:
detailed_listings['last_scraped'].min()

In [ ]:
detailed_listings['last_scraped'].max()

Then plotted and calculated numbers by date. where 8789 out of 8790 (99,99%) comes from 2019-11-09 and single record from 2019-12-06

In [ ]:
detailed_listings.groupby(['last_scraped']).id.count().plot.bar(x='lab', y='val', rot=0)

In [ ]:
detailed_listings.groupby(['last_scraped']).id.count()

In [ ]:
detailed_listings[detailed_listings['last_scraped'] == "2019-12-06"]

In [ ]:
detailed_listings[detailed_listings['id'] == 31640251]

In [ ]:
detailed_listings["days_in_platform"] = (detailed_listings['last_scraped'] - detailed_listings['host_since']).dt.days
detailed_listings['days_in_platform'].fillna(0, inplace=True)


#### 2.1.3. Evaluate special amenities

In [ ]:
results = Counter()
detailed_listings['amenities'].str.strip('{}')\
               .str.replace('"', '')\
               .str.lstrip('\"')\
               .str.rstrip('\"')\
               .str.split(',')\
               .apply(results.update)

results.most_common(10)

In [ ]:
special_needs = pd.DataFrame(results.most_common(1000), columns=['amenity', 'count'])

In [ ]:
special_needs.head(30).sort_values(by = ['count'], ascending = True).plot(kind='barh', x='amenity', y='count',  
                                                      figsize = (10,7), legend = False, color = 'green',
                                                      title = 'description')
plt.xlabel('Count');

Extract special amenities which are the most common in listings

In [ ]:
detailed_listings['Heating'] = detailed_listings['amenities'].str.contains('Heating')
detailed_listings['Essentials'] = detailed_listings['amenities'].str.contains('Essentials')
detailed_listings['Wifi'] = detailed_listings['amenities'].str.contains('Wifi')
detailed_listings['Hair_dryer'] = detailed_listings['amenities'].str.contains('Hair dryer')


##### 2.2. Listings share by neighbourhood. The majority of listings is located in the island, as it contributes to total by 81.3%

In [ ]:
neighboorhood = detailed_listings.groupby("neighbourhood_group_cleansed").agg({'id' :'nunique'})
neighboorhood.rename(columns = {"id": "count"})

In [ ]:
ax = detailed_listings["neighbourhood_group_cleansed"].value_counts().plot(kind='bar',
                                    figsize=(14,8),
                                    title="Number by neighboorhood")
ax.set_xlabel("Neighboorhod")
ax.set_ylabel("Frequency")

#### 2.4.Number of accomodation by room_type

In [ ]:
detailed_listings['property_type'].value_counts().sort_values().plot(kind = 'barh', color = 'green')
plt.title("Number of Accomodation by Property Type")

####  2.5. What is the mean, median of price by property type?

In [ ]:
ax = sns.boxplot(x = "property_type", y = "price", data = detailed_listings)
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)


#### 2.6. Create Venice map with listing location

In [ ]:
#create a map
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        radius=4,
                        weight=0,#remove outline
                        popup = point.name,
                        fill_color='#blue').add_to(this_map)

#use df.apply(,axis=1) to iterate through every row in your dataframe
listings[['latitude', 'longitude', 'name']].apply(plotDot, axis = 1)


#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save(('venice_map.html'))

# this_map

In [ ]:
from IPython.display import HTML

HTML(filename='venice_map.html')

In [ ]:
HTML(filenaxme='venice_map.html')

#### 2.7. When is the peak season?

In [ ]:
available_listings = calendar_all[calendar_all['available'] == 't']

In [ ]:
# Add weekday of available listings
available_listings['weekday'] = available_listings['date'].dt.weekday_name

In [ ]:
available_listings.columns

In [ ]:
# remove_outliers
col_list_dates = ["price"]
remove_outliers(available_listings, col_list_dates)

In [ ]:
ax = sns.boxplot(x = "weekday", y = "price", data = remove_outliers(available_listings, col_list_dates), order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)


In [ ]:
unavailable_listings = calendar_all[calendar_all['available'] == 'f']
unavailable_listings['weekday'] = unavailable_listings['date'].dt.weekday_name

In [ ]:
ax = sns.boxplot(x = "weekday", y = "price", data = remove_outliers(unavailable_listings, col_list_dates), order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
ax.set_xticklabels(ax.get_xticklabels(), rotation = 90)

##### Distributions of data

In [ ]:
int_detailed_listings = detailed_listings.select_dtypes(include=['int64'])

In [ ]:
fig, ax = plt.subplots(figsize=(19,10), dpi=50)
int_detailed_listings.hist(ax=ax, layout=(7,7), alpha=0.5)

In [ ]:
detailed_listings.drop(['id', 'host_id'], axis = 1).describe()

#### 2.9 How clients review their stay?

In [ ]:
# Need to download stopwords
import nltk
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words(['english', 'italian', 'french'])) 

In [ ]:
# Add location values in italian and english
stop_words.update(["Venice", "Venezia"])
stop_words.update(["us", "gave", "also"])
stop_words.update(["de", "la"])

In [ ]:
detailed_reviews["comments"].fillna("no review", inplace = True)

In [ ]:
text = " ".join(review for review in detailed_reviews.comments)

In [ ]:
eye_mask = np.array(Image.open("eye_mask.png"))
eye_mask

In [ ]:
def transform_format(val):
    if val == 0:
        return 255
    else:
        return val

In [ ]:
transformed_eyes_mask = np.ndarray((eye_mask.shape[0],eye_mask.shape[1]), np.int32)

for i in range(len(eye_mask)):
    transformed_eyes_mask[i] = list(map(transform_format, eye_mask[i]))
transformed_eyes_mask

In [ ]:
# wc = WordCloud(max_font_size = 50, max_words = 150, background_color = "white", stopwords = stop_words, mask = eye_mask).generate(text)
wc = WordCloud(max_font_size = 100, background_color="white", max_words= 200, mask=transformed_eyes_mask, stopwords = stop_words, contour_width=3, contour_color='black').generate(text)

In [ ]:
wc.to_file("mask.png")

In [ ]:
wc

In [ ]:
plt.figure(figsize=[20,10])
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
detailed_listings

#### 2.10 What is the relationship between location and price?

In [ ]:
plt.figure(figsize=(6,6))
sns.heatmap(detailed_listings.groupby(['neighbourhood', 'room_type']).price.median().unstack(), 
            cmap='Reds', annot=True, fmt=".0f")

plt.xlabel('\nRoom Type', fontsize=12)
plt.ylabel('District\n', fontsize=12)
plt.title('\nHeatmap: Median Prices by room type and district\n\n', fontsize=14, fontweight='bold');

#### 2.11 Which KPIS the most affect price?

In [ ]:
# create temp df to avoid duplicate columns
temp_listings = listings[['id', 'listings_km_bins']]

In [ ]:
new_df = pd.merge(detailed_listings, temp_listings,  how='left', left_on=['id'], right_on = ['id'])


In [ ]:
selected_cols = ['price', 'bathrooms', 'cleaning_fee', 
        'host_is_superhost',
       'host_total_listings_count',
       'host_identity_verified',
#        'property_type',
                 'room_type',
       'bedrooms', 'beds', 
       'minimum_nights', 'maximum_nights',
       'number_of_reviews', 'review_scores_rating',
       'days_in_platform', 'Heating', 'Essentials', 'Wifi', 'Hair_dryer', 'listings_km_bins']

In [ ]:
regression_df = new_df[selected_cols]
regression_df['listings_km_bins'] = regression_df.listings_km_bins.astype(str)

regression_df.columns

In [ ]:
# sns.pairplot(regression_df)

In [ ]:
bool_cols = list(regression_df.select_dtypes(include='bool').columns)
text_cols = list(regression_df.select_dtypes(include='object').columns)
float_cols = list(regression_df.select_dtypes(include='float').columns)
cols_ = bool_cols + text_cols
regression_df[float_cols] = regression_df[float_cols].astype(int)

regression = pd.get_dummies(regression_df, columns = cols_)

In [ ]:
X = regression.drop("price", axis = 1)
y = regression.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm

In [ ]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])  
coeff_df.sort_values(by="Coefficient", ascending = False)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
r2 = r2_score(y_test, y_pred)

In [ ]:
r2